In [1]:
'''Step 1:
  Conacanate AFDC historical data files into a single DataFrame.
    Load cleaned geojson files into GeoDataFrames.
'''
#Loading cleaned data
import pandas as pd
import geopandas as gpd
import os
import glob

processed_path = "../data/processed/"
afdc_files = glob.glob(os.path.join(processed_path, "alt_fuel_stations_historical_day*.csv"))

if not afdc_files:
    raise FileNotFoundError("No AFDC historical data files found in the processed data directory.")

#Create a list to hold each year's DataFrame
else:
    afdc_df_list = []
    for file in afdc_files:
        try:
            year = int(file.split('(')[-1].split(')')[0].split(' ')[-1])
            temp_df = pd.read_csv(file)
            temp_df['Year'] = year
            afdc_df_list.append(temp_df)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

#Combine all years into single DataFrame
if afdc_df_list:
    afdc_data = pd.concat(afdc_df_list, ignore_index=True)
    print(f"Successfully loaded and combined {len(afdc_files)} AFDC files into a single DF.")
    print(f"Total records: {len(afdc_data)}")
    print(f"Years found: {sorted(afdc_data['Year'].unique())}")
else:
    
    print("Warning: No data was loaded into the afdc_df_list.")
    afdc_data = pd.DataFrame()

#Saving loaded data to a single file for easier access in future steps
combined_afdc_path = "../data/processed/combined_afdc_data.csv" 
try:
    afdc_data.to_csv(combined_afdc_path, index=False)
    print(f"Combined AFDC data saved to {combined_afdc_path}")
except Exception as e:
    print(f"Error saving combined AFDC data: {e}")

#Load geojson data
try:
    gdf_chargers = gpd.read_file(os.path.join(processed_path, "chargingStationWashington_cleaned.geojson"))
    gdf_roads = gpd.read_file(os.path.join(processed_path, "majorRoadsWashington_cleaned.geojson"))
    gdf_shops = gpd.read_file(os.path.join(processed_path, "shopsWashington_cleaned.geojson"))
    gdf_leisure = gpd.read_file(os.path.join(processed_path, "leisureWashington_cleaned.geojson"))
    gdf_residential = gpd.read_file(os.path.join(processed_path, "residentialWashington_cleaned.geojson"))
    gdf_amenities = gpd.read_file(os.path.join(processed_path, "amenitiesWashington_cleaned.geojson"))


    print(f"Loaded {len(gdf_chargers)} charging stations.")
    print(f"Loaded {len(gdf_roads)} road segments.")
    print(f"Loaded {len(gdf_shops)} shops.")
    print(f"Loaded {len(gdf_leisure)} leisure spots.")
    print(f"Loaded {len(gdf_residential)} residential buildings.")
    print(f"Loaded {len(gdf_amenities)} amenities.")

except FileNotFoundError as e:
    print(f"Error loading files: {e}. Make sure all cleaned files are in the 'processed' directory.")

Successfully loaded and combined 11 AFDC files into a single DF.
Total records: 16721
Years found: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Combined AFDC data saved to ../data/processed/combined_afdc_data.csv
Loaded 676 charging stations.
Loaded 47785 road segments.
Loaded 29895 shops.
Loaded 53416 leisure spots.
Loaded 125104 residential buildings.
Loaded 145434 amenities.


In [2]:
poi_list = [gdf_shops, gdf_leisure, gdf_residential, gdf_amenities]
gdf_pois = pd.concat(poi_list, ignore_index=True)
print(f"Combined all POIs into a single dataframe with {len(gdf_pois)} records.")

#Set CRS (coordinate reference system) to ESPG:32148, Washington's (NAD83) projected CRS
target_crs = "EPSG:32148"
gdf_chargers = gdf_chargers.to_crs(target_crs)
gdf_roads = gdf_roads.to_crs(target_crs)
gdf_pois = gdf_pois.to_crs(target_crs)

print("All GeoDataFrames reprojected to EPSG:32148.")


Combined all POIs into a single dataframe with 353849 records.
All GeoDataFrames reprojected to EPSG:32148.


In [3]:
"""
Calculating proximity and density metrics, using TQDM, cool side node
tqdm's name derives from the Arabic word taqadum which means progress because
tqdm is a fast, extensible progress bar for Python and CLI."""
from tqdm.notebook import tqdm
import geopandas as gpd
import pandas as pd

# --- Step 1: Combine all POI GeoDataFrames into one ---
# This makes it much easier to calculate the density for all POI types at once.
# This step fixes the "gdf_pois is not defined" error.
poi_list = [gdf_shops, gdf_leisure, gdf_residential, gdf_amenities]
gdf_pois = pd.concat(poi_list, ignore_index=True)

print(f"--- POI Combination Complete ---")
print(f"Combined all POIs into a single GeoDataFrame with {len(gdf_pois)} features.")


# --- Step 2: Re-project all GeoDataFrames to a Projected CRS ---
# This step fixes the UserWarning and ensures your distance calculations are accurate (in meters).
target_crs = "EPSG:32148" # A good projected CRS for Washington

print(f"\n--- Re-projecting all GeoDataFrames to {target_crs} for accurate calculations ---")
gdf_chargers = gdf_chargers.to_crs(target_crs)
gdf_roads = gdf_roads.to_crs(target_crs)
gdf_pois = gdf_pois.to_crs(target_crs)
print("Re-projection complete.")


# --- Step 3: Calculate Proximity and Density Metrics ---
tqdm.pandas(desc="Calculating distances")  # Enable tqdm for pandas

major_road_types = ['motorway', 'trunk', 'primary', 'secondary']
gdf_major_roads = gdf_roads[gdf_roads['highway'].isin(major_road_types)]
print(f"\nFiltered roads down to {len(gdf_major_roads)} major road segments.")

print("\nCalculating distance to nearest major road...")
"""
Code below works like this. Firstly, we add a new column 'dist_to_major_road_m' to gdf_chargers.
then we select the geometry key from the gdf_chargers GeoDataFrame, which contains the point geometries of the charging stations.
the line progress apply tells pandas to apply a defined function to each point in the geometry column, while also displaying a progress bar using tqdm.
lambda point is like an anonymous mini-function that you dont make a funtion for. Then it calculates the distance from point(the charging station)
to all geometries in gdf_major_roads using gdf_major_roads.distance(point). min function is there since the function finds distances to all roads
in the dataset and we only want the minimum distance, which is the nearest road.

"""
gdf_chargers['dist_to_major_road_m'] = gdf_chargers.geometry.progress_apply(
    lambda point: gdf_major_roads.distance(point).min()
)
print("Distance calculation complete.")

print("\nCalculating POI density within 500 meters...")
# Step 1: Create a 500-meter circular buffer around each charging station
buffers = gdf_chargers.geometry.buffer(500)

# Step 2: Perform a spatial join to find all POIs that fall within each buffer
"""
 Explanation of spatial join:
 the left gdf is gdf_pois, which contains the point geometries of the POIs. then the right
 gdf is each charging station with the 500m buffer around it. The join is an inner join
 meaning that POIs thast fall within the 500m buffer of any charging station will be kept in the result.
 The predicate "within" specifies that we are looking for POIs that are located within the buffer areas.
 
"""
joined_pois = gpd.sjoin(gdf_pois, gpd.GeoDataFrame(geometry=buffers), how="inner", predicate="within")

# Step 3: Group by the charging station's index and count the number of POIs
poi_counts = joined_pois.groupby(joined_pois.index).size()

# Step 4: Add the counts to your main DataFrame and fill empty values with 0
gdf_chargers['poi_density_500m'] = poi_counts
gdf_chargers['poi_density_500m'] = gdf_chargers['poi_density_500m'].fillna(0)
print("POI density calculation complete.")


# --- Step 4: Display and Save Results ---
print("\n--- Feature Engineering Complete ---")
print("first 5 rows of gdf_chargers with new features:")
print(gdf_chargers[['dist_to_major_road_m', 'poi_density_500m']].head())

print("\nSaving updated GeoDataFrame...")
output_path = "../data/processed/stations_with_features.geojson"
try:
    gdf_chargers.to_file(output_path, driver='GeoJSON')
    print(f"Updated GeoDataFrame saved to {output_path}")
except Exception as e:
    print(f"Error saving updated GeoDataFrame: {e}")

--- POI Combination Complete ---
Combined all POIs into a single GeoDataFrame with 353849 features.

--- Re-projecting all GeoDataFrames to EPSG:32148 for accurate calculations ---
Re-projection complete.

Filtered roads down to 47785 major road segments.

Calculating distance to nearest major road...


Calculating distances:   0%|          | 0/676 [00:00<?, ?it/s]

Distance calculation complete.

Calculating POI density within 500 meters...
POI density calculation complete.

--- Feature Engineering Complete ---
first 5 rows of gdf_chargers with new features:
   dist_to_major_road_m  poi_density_500m
0             32.676352               2.0
1           1887.571780               3.0
2            156.935476               0.0
3             74.914555               1.0
4            374.378686               0.0

Saving updated GeoDataFrame...
Updated GeoDataFrame saved to ../data/processed/stations_with_features.geojson


In [7]:
import pandas as pd
import geopandas as gpd
import os
from tqdm.notebook import tqdm




# Drop duplicate station IDs to get unique stations, keeping the first entry
afdc_unique = afdc_data.drop_duplicates(subset=['ID'], keep='first').copy()
print(f"Found {len(afdc_unique)} unique stations from the AFDC dataset.")

# Convert the unique AFDC DataFrame to a GeoDataFrame
gdf_afdc_unique = gpd.GeoDataFrame(
    afdc_unique,
    geometry=gpd.points_from_xy(afdc_unique.Longitude, afdc_unique.Latitude),
    crs="EPSG:4326"  # Standard CRS for lat/lon
)

# Combine all POIs into a single GeoDataFrame
poi_list = [gdf_shops, gdf_leisure, gdf_residential, gdf_amenities]
gdf_pois = pd.concat(poi_list, ignore_index=True)
print(f"Combined {len(gdf_pois)} points of interest.")

# Re-project all data to a common Projected CRS for accurate distance calculation
target_crs = "EPSG:32148"
print(f"Re-projecting all data to {target_crs} for accurate meter-based calculations")
gdf_afdc_unique = gdf_afdc_unique.to_crs(target_crs)
gdf_roads = gdf_roads.to_crs(target_crs)
gdf_pois = gdf_pois.to_crs(target_crs)
print("Re-projection complete.")


# Calculate Distance to Nearest Major Road 
tqdm.pandas(desc="Calculating road distances")
# Filter for major road types
major_road_types = ['motorway', 'trunk', 'primary', 'secondary']
gdf_major_roads = gdf_roads[gdf_roads['highway'].isin(major_road_types)]

# Calculate the minimum distance from each station to any major road
print(f"Calculating distance to nearest of {len(gdf_major_roads)} major road segments...")
gdf_afdc_unique['dist_to_major_road_m'] = gdf_afdc_unique.geometry.progress_apply(
    lambda point: gdf_major_roads.distance(point).min()
)
print("Distance calculation complete.")

# --- 5. Calculate POI Density within 500m ---
print("Calculating POI density within a 500-meter radius")
# Create a 500m buffer around each unique station
buffers = gdf_afdc_unique.geometry.buffer(500)

# Spatially join the POIs that are within each buffer
joined_pois = gpd.sjoin(gdf_pois, gpd.GeoDataFrame(geometry=buffers), how="inner", predicate="within")

# Group by the station index and count the number of POIs in each buffer
poi_counts = joined_pois.groupby(joined_pois.index).size()

# Add the counts back to the main DataFrame, filling stations with no nearby POIs with 0
gdf_afdc_unique['poi_density_500m'] = poi_counts
gdf_afdc_unique['poi_density_500m'] = gdf_afdc_unique['poi_density_500m'].fillna(0)
print("POI density calculation complete.")


# --- 6. Display and Save Results ---
print("\\n--- Feature Engineering Complete ---")
print("Sample of unique AFDC stations with new features:")
print(gdf_afdc_unique[['ID', 'Open Date', 'dist_to_major_road_m', 'poi_density_500m']].head())

output_path = os.path.join(processed_path, "afdc_unique_stations_with_features.geojson")
try:
    gdf_afdc_unique.to_file(output_path, driver='GeoJSON')
    print(f"\\nSuccessfully saved updated GeoDataFrame to {output_path}")
except Exception as e:
    print(f"\\nError saving file: {e}")




Found 4718 unique stations from the AFDC dataset.
Combined 353849 points of interest.
Re-projecting all data to EPSG:32148 for accurate meter-based calculations
Re-projection complete.
Calculating distance to nearest of 47785 major road segments...


Calculating road distances:   0%|          | 0/4718 [00:00<?, ?it/s]

Distance calculation complete.
Calculating POI density within a 500-meter radius
POI density calculation complete.
\n--- Feature Engineering Complete ---
Sample of unique AFDC stations with new features:
     ID Open Date  dist_to_major_road_m  poi_density_500m
0   134       NaN            332.932701               8.0
1  1160       NaN             61.177862               3.0
2  1170       NaN             57.522281               0.0
3  1200       NaN              2.257608               2.0
4  1201       NaN            184.838663               0.0
\nSuccessfully saved updated GeoDataFrame to ../data/processed/afdc_unique_stations_with_features.geojson


In [4]:
import pandas as pd

df = pd.read_csv("../data/processed/combined_afdc_data.csv")

#getting count of chargers from year 2025
count_2025 = df[df['Year'] == 2025].shape[0]
print(f"Number of chargers from year 2025: {count_2025}")

Number of chargers from year 2025: 2979
